In [2]:
#This is Maria. Week 3 Capstone Project: Segmenting and Clustering Neighborhoods in Toronto
# Libraries:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup
import urllib

In [3]:
#get file from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

response = urllib.request.urlopen(url)         # get the content 
webContent = response.read()                   # read the content 
soup = BeautifulSoup (webContent,'lxml')       # ready for parsing

In [4]:
relevant_html_table = soup.find('table',class_="wikitable sortable")


toronto_neighborhood_df_columns = []
# extract the feature names from the column headings of the table 
# hint: look for 'th' in the html

for table_row in relevant_html_table.find_all('tr'):
    for table_col in table_row.find_all('th'):
        toronto_neighborhood_df_columns.append(table_col.text.strip())

# uncomment the following line if you want to see intermediate results
print('For the dataframe the column headings will be: ', list(toronto_neighborhood_df_columns))

For the dataframe the column headings will be:  ['Postcode', 'Borough', 'Neighbourhood']


In [5]:
neighborhood_array = []                                # declare a null array to gather postcode, borough and neighborhood details

for table_row in relevant_html_table.find_all('tr'):
    row_postcode     = ''
    row_borough      = ''
    row_neighborhood = ''
    col_count        = 0
    
    for table_col in table_row.find_all('td'):
        col_count    += 1
        if col_count == 4: 
            print ("Error! Too many columns! Unexpected column encountered.")
            break
        if col_count == 1: row_postcode = table_col.text.strip()
        if col_count == 2: row_borough  = table_col.text.strip()
        if col_count == 3: 
            row_neighborhood   = table_col.text.strip()

In [6]:
# data go to a dataframe
toronto_neighborhood_df = pd.DataFrame (data=neighborhood_array,columns=list(toronto_neighborhood_df_columns))

In [7]:
toronto_neighborhood_df.head(3)

,Postcode,Borough,Neighbourhood


In [19]:
toronto_neighborhood_df.shape

(288, 3)

In [20]:
toronto_neighborhood_df[toronto_neighborhood_df['Neighbourhood'] =='Not assigned']

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned
13,M2B,Not assigned,Not assigned
20,M7B,Not assigned,Not assigned
21,M8B,Not assigned,Not assigned
30,M2C,Not assigned,Not assigned
36,M7C,Not assigned,Not assigned
37,M8C,Not assigned,Not assigned


In [21]:
for i in range(toronto_neighborhood_df.shape[0]):
    if toronto_neighborhood_df.at [i,'Borough']       != 'Not assigned' and \
       toronto_neighborhood_df.at [i,'Neighbourhood'] == 'Not assigned':
        print ("Before values for index {:4d} are: {:20s} {:20s} {:20s}" \
               .format(i, toronto_neighborhood_df.at[i,"Postcode"],      \
                       toronto_neighborhood_df.at[i,"Borough"],          \
                       toronto_neighborhood_df.at[i,"Neighbourhood"]))
        toronto_neighborhood_df.at [i,'Neighbourhood'] = toronto_neighborhood_df.at [i,'Borough']
        print ("After  values for index {:4d} are: {:20s} {:20s} {:20s}" \
               .format(i, toronto_neighborhood_df.at[i,"Postcode"],      \
                       toronto_neighborhood_df.at[i,"Borough"],          \
                       toronto_neighborhood_df.at[i,"Neighbourhood"]))

Before values for index    8 are: M7A                  Queen's Park         Not assigned        
After  values for index    8 are: M7A                  Queen's Park         Queen's Park        


In [22]:
toronto_neighborhood_df.groupby (['Borough'])['Borough'].count()

Borough
Central Toronto     17
Downtown Toronto    37
East Toronto         7
East York            6
Etobicoke           45
Mississauga          1
North York          38
Not assigned        77
Queen's Park         1
Scarborough         37
West Toronto        13
York                 9
Name: Borough, dtype: int64

In [23]:
print ("There are {} rows with a Borough name of Not assigned" 
       .format(toronto_neighborhood_df[toronto_neighborhood_df['Borough']=='Not assigned']['Borough'].count()))

There are 77 rows with a Borough name of Not assigned


In [24]:
# Create a list 
indexes_to_delete = toronto_neighborhood_df [toronto_neighborhood_df ['Borough'] == 'Not assigned'].index
print (indexes_to_delete)

Int64Index([  0,   1,   9,  13,  20,  21,  30,  36,  37,  45,  46,  50,  51,
             52,  54,  55,  59,  60,  61,  73,  74,  75,  88,  89,  90, 104,
            105, 106, 120, 121, 136, 137, 148, 149, 155, 161, 162, 167, 175,
            181, 182, 188, 189, 190, 194, 195, 201, 202, 203, 204, 209, 210,
            223, 224, 237, 238, 241, 242, 247, 248, 253, 254, 258, 259, 260,
            261, 263, 264, 274, 275, 276, 277, 278, 279, 280, 281, 287],
           dtype='int64')


In [25]:
# use the list of indexes and delete those rows from the data frame
# reset the row indexes
toronto_neighborhood_df.drop(index=indexes_to_delete, inplace=True)
toronto_neighborhood_df.reset_index(drop=True, inplace=True)
toronto_neighborhood_df.shape

(211, 3)

In [26]:
toronto_neighborhood_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [28]:
toronto_neighborhood_df[toronto_neighborhood_df['Neighbourhood'] =='Not assigned']

,Postcode,Borough,Neighbourhood


In [29]:
# sort the table to make sure the values are sorted by Postcode+Borough+Neighborhood
toronto_neighborhood_df.sort_values(['Postcode', 'Borough','Neighbourhood'], ascending=[True,True,True],axis=0,inplace=True)
toronto_neighborhood_df.reset_index(drop=True,inplace=True)

In [30]:
# we will later verify that all these 3 neighborhoods were consolidated against one row for M1K - Scarborough
toronto_neighborhood_df[toronto_neighborhood_df['Postcode']=='M1K']

,Postcode,Borough,Neighbourhood
11,M1K,Scarborough,East Birchmount Park
12,M1K,Scarborough,Ionview
13,M1K,Scarborough,Kennedy Park


In [31]:
toronto_neighborhood_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern
1,M1B,Scarborough,Rouge
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Port Union
4,M1C,Scarborough,Rouge Hill


In [33]:
grouped_neighborhood_array = []                                # declare a null array
row_postcode     = ''
row_borough      = ''
row_neighborhood = ''
very_first = True

for i in range(toronto_neighborhood_df.shape[0]):
# uncomment the following line(s) if you want to see intermediate results
#    print ("Just now read a row {:3d} {:10s} {:10s} {:10s}" \
#           .format (i, 
#                    toronto_neighborhood_df.at[i,"Postcode"],
#                    toronto_neighborhood_df.at[i,"Borough"],
#                    toronto_neighborhood_df.at[i,"Neighbourhood"])
#          )

# execute the logic when there is a change in postcode or 
# the combination of postcode and borough has changed
    if (toronto_neighborhood_df.at [i,'Postcode'] != row_postcode) or \
       (toronto_neighborhood_df.at [i,'Postcode'] == row_postcode  and \
        toronto_neighborhood_df.at [i,'Borough']  != row_borough):
# uncomment the following line(s) if you want to see intermediate results
#        print ("Prev values are:         {:20s} {:20s} {:20s}" \
#               .format(row_postcode,  row_borough,  row_neighborhood))
        if very_first:
            very_first = False
        else:
            grouped_neighborhood_array.append ([row_postcode, row_borough,row_neighborhood]) 
# uncomment the following line(s) if you want to see intermediate results
#        print ("New  values are:         {:20s} {:20s} {:20s}" \
#               .format(toronto_neighborhood_df.at[i,"Postcode"],
#                       toronto_neighborhood_df.at[i,"Borough"],
#                       toronto_neighborhood_df.at[i,"Neighbourhood"]))
        row_postcode     = toronto_neighborhood_df.at [i,'Postcode']
        row_borough      = toronto_neighborhood_df.at [i,'Borough']
        row_neighborhood = toronto_neighborhood_df.at[i,"Neighbourhood"]
    else:
#       if the postcode and borough are the same then get all neighborhoods into one field
        row_neighborhood += ", "
        row_neighborhood += toronto_neighborhood_df.at[i,"Neighbourhood"]
# uncomment the following line if you want to see intermediate results
#        print ("Concatenated values is : {:20s} {:20s} {:20s} " .format(row_postcode, row_borough, row_neighborhood))


grouped_neighborhood_array.append ([row_postcode, row_borough,row_neighborhood])

In [36]:
grouped_toronto_neighborhood_df = pd.DataFrame (data=grouped_neighborhood_array,columns=list(toronto_neighborhood_df_columns))
grouped_toronto_neighborhood_df.head(15)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [38]:
grouped_toronto_neighborhood_df[grouped_toronto_neighborhood_df['Postcode']=='M1K']

,Postcode,Borough,Neighbourhood
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"


In [39]:
grouped_toronto_neighborhood_df.shape

(103, 3)

In [40]:

print('The Toronto dataframe has {} boroughs and {} neighborhoods.'.format(
        len(grouped_toronto_neighborhood_df['Borough'].unique()),
        grouped_toronto_neighborhood_df.shape[0]
    )
)

The Toronto dataframe has 11 boroughs and 103 neighborhoods.


In [41]:
can_geo_cords_url = 'https://cocl.us/Geospatial_data'
can_geo_cord_df = pd.read_csv(can_geo_cords_url)
can_geo_cord_df.shape

(103, 3)

In [42]:
can_geo_cord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
grouped_toronto_neighborhood_with_latlong_df = pd.concat([grouped_toronto_neighborhood_df, can_geo_cord_df], axis=1)
grouped_toronto_neighborhood_with_latlong_df.shape

(103, 6)

In [45]:
grouped_toronto_neighborhood_with_latlong_df = pd.merge(left=grouped_toronto_neighborhood_df,
                                                        right=can_geo_cord_df, 
                                                        left_on='Postcode', 
                                                        right_on='Postal Code')
grouped_toronto_neighborhood_with_latlong_df.drop('Postal Code',axis = 1, inplace=True)
grouped_toronto_neighborhood_with_latlong_df.shape

(103, 5)

In [46]:
grouped_toronto_neighborhood_with_latlong_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [47]:
grouped_toronto_neighborhood_with_latlong_df.tail()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
102,M9W,Etobicoke,Northwest,43.706748,-79.594054


In [48]:
grouped_toronto_neighborhood_with_latlong_df.groupby('Borough').count()

,Postcode,Neighbourhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,18,18,18,18
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Queen's Park,1,1,1,1
Scarborough,17,17,17,17


In [49]:
specific_postal_codes = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A']
specific_postal_codes_df = pd.DataFrame (data=specific_postal_codes,columns={'Postcode'})
specific_postal_codes_df.head(5)

,Postcode
0,M5G
1,M2H
2,M4B
3,M1J
4,M4G


In [50]:
specific_toronto_neighborhood_with_latlong_df = pd.merge(left=grouped_toronto_neighborhood_with_latlong_df,right=specific_postal_codes_df, left_on='Postcode', right_on='Postcode')
specific_toronto_neighborhood_with_latlong_df.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
2,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
3,M2H,North York,Hillcrest Village,43.803762,-79.363452
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937


In [51]:
Toronto_Latitude  = grouped_toronto_neighborhood_with_latlong_df['Latitude'].mean()
Toronto_Longitude = grouped_toronto_neighborhood_with_latlong_df['Longitude'].mean()
print ("(just for comparison) \nHere are Toronto's \nLatitude {} and Longitude {}\n\
computed using the mean value of the available \
longitudes and Latitudes" .format(Toronto_Latitude,Toronto_Longitude))

(just for comparison) 
Here are Toronto's 
Latitude 43.70460773398059 and Longitude -79.39715291165048
computed using the mean value of the available longitudes and Latitudes


In [52]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # This will convert a given address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [53]:
import json 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


branca-0.3.1         | 25 KB     | #################################

In [55]:
# Let us analyze neighborhoods of only selected Borough(s)
Borough_to_analyze_contains = 'Toro'              # <<< make your choice here
list_of_boroughs = grouped_toronto_neighborhood_with_latlong_df[grouped_toronto_neighborhood_with_latlong_df.Borough.str.contains(Borough_to_analyze_contains)]['Borough'].unique()
print ('We will analyze the following Boroughs:',list_of_boroughs)

We will analyze the following Boroughs: ['East Toronto' 'Central Toronto' 'Downtown Toronto' 'West Toronto']


In [56]:
selected_borough_df = grouped_toronto_neighborhood_with_latlong_df [grouped_toronto_neighborhood_with_latlong_df.Borough.str.contains(Borough_to_analyze_contains)]
selected_borough_df.reset_index(inplace=True,drop=True)
selected_borough_df.shape

(38, 5)

In [57]:
print ("Here are the {} neighborhoods within the selected {} boroughs: \n{}" .format(
    len(selected_borough_df['Neighbourhood'].unique()), 
    len(selected_borough_df['Borough'].unique()),
    selected_borough_df['Neighbourhood'].unique()))

Here are the 38 neighborhoods within the selected 4 boroughs: 
['The Beaches' 'Riverdale, The Danforth West'
 'India Bazaar, The Beaches West' 'Studio District' 'Lawrence Park'
 'Davisville North' 'North Toronto West' 'Davisville'
 'Moore Park, Summerhill East'
 'Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West'
 'Rosedale' 'Cabbagetown, St. James Town' 'Church and Wellesley'
 'Harbourfront, Regent Park' 'Garden District, Ryerson' 'St. James Town'
 'Berczy Park' 'Central Bay Street' 'Adelaide, King, Richmond'
 'Harbourfront East, Toronto Islands, Union Station'
 'Design Exchange, Toronto Dominion Centre'
 'Commerce Court, Victoria Hotel' 'Roselawn'
 'Forest Hill North, Forest Hill West'
 'North Midtown, The Annex, Yorkville' 'Harbord, University of Toronto'
 'Chinatown, Grange Park, Kensington Market'
 'Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara'
 'Stn A PO Boxes 25 The Esplanade'
 'First Canadian Place, U

In [58]:
selected_borough_df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [60]:
# here are my personal Foursquare credentials

CLIENT_ID     = 
CLIENT_SECRET = 

In [61]:
VERSION = '20180604'
radius_mts=500
search_LIMIT=100

In [67]:
def getNeighVenuesofInterest (i, my_df):
    venues_in_neighborhood = []
    Lati_of_interest    = my_df.loc[i]['Latitude']
    Long_of_interest    = my_df.loc[i]['Longitude']
    Burough_of_interest = my_df.loc[i]['Borough']
    Neigh_of_interest   = my_df.loc[i]['Neighbourhood']

# uncomment the following line if you want to see intermediate results    
#    print ("Values from this row are \nBorough: {:15s} Neighborhood: {:30s} Latitude: {:.5f}   Longitude: {:.5f}" \
#       .format(Burough_of_interest,
#               Neigh_of_interest,
#               Lati_of_interest,
#               Long_of_interest))
    
    
    # Explore and get a bunch of venues within a slected radius
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, Lati_of_interest, Long_of_interest, VERSION,radius_mts, search_LIMIT)
    results = requests.get(url).json()['response']['groups'][0]['items']
# uncomment the following line if you want to see intermediate results
#    print ("Here are interesting venues for {}\'s neighborhood(s) of {}" .format(Burough_of_interest,Neigh_of_interest))
    for each_occ in results:
        venue_cat  = each_occ ['venue']['categories'][0]['name']
        venue_name = each_occ ['venue']['name']
        venue_lat  = each_occ ['venue']['location']['lat']
        venue_lng  = each_occ ['venue']['location']['lng']
# uncomment the following line if you want to see intermediate results
#       print ("{:25s} {:35s} {:5f} {:5f}" .format(venue_cat, venue_name, venue_lat, venue_lng))
        venues_in_neighborhood.append ([Burough_of_interest, 
                                        Neigh_of_interest,
                                        Lati_of_interest,
                                        Long_of_interest,
                                        venue_cat,
                                        venue_name,
                                        venue_lat,
                                        venue_lng])
#   print("Venues in neighborhood are:", venues_in_neighborhood)    # uncomment if you want to see intermediate results
    return(venues_in_neighborhood)


In [68]:
venues_df  = pd.DataFrame(columns=['Burough', 'Neighborhood', 'Neigh_Lat', 'Neigh_Long', 'Venue_Cat', 'Venue_Name', 'Venue_Lat', 'Venue_Long'])
venues_df.shape

(0, 8)

In [69]:
venues_df.head()

,Burough,Neighborhood,Neigh_Lat,Neigh_Long,Venue_Cat,Venue_Name,Venue_Lat,Venue_Long


In [70]:
# in a loop get the interesting venues for each neighborhood (1N=one neighborhood)
# accumulate the details for each neighborhood (1N dataframe) into the consolidated venues_df dataframe

for i in range(selected_borough_df.shape[0]):
    venues_1N_df = []
    interesting_venues = []
    interesting_venues = getNeighVenuesofInterest(i,selected_borough_df)
    venues_1N_df = pd.DataFrame (data=interesting_venues, 
                                 columns=['Burough', 'Neighborhood', 'Neigh_Lat', 'Neigh_Long', 
                                          'Venue_Cat', 'Venue_Name', 'Venue_Lat', 'Venue_Long'])
    venues_df = pd.concat([venues_df, venues_1N_df],axis=0)
    venues_df.reset_index(drop=True,inplace=True)
#   uncomment the lines below to see how the venues_df is growing
    print("The venues_df dataframe has {:4d} venue rows and {:2d} columns" 
          .format(venues_df.shape[0], venues_df.shape[1]))

The venues_df dataframe has    4 venue rows and  8 columns
The venues_df dataframe has   46 venue rows and  8 columns
The venues_df dataframe has   65 venue rows and  8 columns
The venues_df dataframe has  104 venue rows and  8 columns
The venues_df dataframe has  107 venue rows and  8 columns
The venues_df dataframe has  116 venue rows and  8 columns
The venues_df dataframe has  140 venue rows and  8 columns
The venues_df dataframe has  175 venue rows and  8 columns
The venues_df dataframe has  177 venue rows and  8 columns
The venues_df dataframe has  192 venue rows and  8 columns
The venues_df dataframe has  196 venue rows and  8 columns
The venues_df dataframe has  240 venue rows and  8 columns
The venues_df dataframe has  328 venue rows and  8 columns
The venues_df dataframe has  379 venue rows and  8 columns
The venues_df dataframe has  479 venue rows and  8 columns
The venues_df dataframe has  579 venue rows and  8 columns
The venues_df dataframe has  635 venue rows and  8 colum

In [71]:
venues_df.shape

(1696, 8)

In [72]:
print ("The {} unique boroughs in the venues dataframe are: \n{}" .format(
    len(venues_df['Burough'].unique()), 
    list(venues_df['Burough'].unique())))

The 4 unique boroughs in the venues dataframe are: 
['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']


In [73]:
print ("The {} unique neighborhoods in the venues dataframe are: \n{}" .format(
    len(venues_df['Neighborhood'].unique()), 
    list(venues_df['Neighborhood'].unique())))

The 38 unique neighborhoods in the venues dataframe are: 
['The Beaches', 'Riverdale, The Danforth West', 'India Bazaar, The Beaches West', 'Studio District', 'Lawrence Park', 'Davisville North', 'North Toronto West', 'Davisville', 'Moore Park, Summerhill East', 'Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West', 'Rosedale', 'Cabbagetown, St. James Town', 'Church and Wellesley', 'Harbourfront, Regent Park', 'Garden District, Ryerson', 'St. James Town', 'Berczy Park', 'Central Bay Street', 'Adelaide, King, Richmond', 'Harbourfront East, Toronto Islands, Union Station', 'Design Exchange, Toronto Dominion Centre', 'Commerce Court, Victoria Hotel', 'Roselawn', 'Forest Hill North, Forest Hill West', 'North Midtown, The Annex, Yorkville', 'Harbord, University of Toronto', 'Chinatown, Grange Park, Kensington Market', 'Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara', 'Stn A PO Boxes 25 The Esplanade', 'First Canadian 

In [74]:
list_of_venue_categories = venues_df['Venue_Cat'].unique()
print ("The {} unique venue categories in the venues dataframe are: \n{}" .format(
    len(list_of_venue_categories), 
    list(list_of_venue_categories)))

The 234 unique venue categories in the venues dataframe are: 
['Trail', 'Health Food Store', 'Pub', 'Neighborhood', 'Greek Restaurant', 'Cosmetics Shop', 'Italian Restaurant', 'Ice Cream Shop', 'Brewery', 'Yoga Studio', 'Fruit & Vegetable Store', 'Pizza Place', 'Juice Bar', 'Dessert Shop', 'Restaurant', 'Bookstore', 'Bubble Tea Shop', 'Furniture / Home Store', 'Spa', 'Diner', 'Grocery Store', 'Indian Restaurant', 'Caribbean Restaurant', 'Coffee Shop', 'Bakery', 'Café', 'American Restaurant', 'Lounge', 'Liquor Store', 'Burger Joint', 'Gym', 'Fish & Chips Shop', 'Park', 'Sushi Restaurant', 'Steakhouse', 'Pet Store', 'Burrito Place', 'Fast Food Restaurant', 'Movie Theater', 'Sandwich Place', 'Food & Drink Shop', 'Fish Market', 'Gay Bar', 'Seafood Restaurant', 'Middle Eastern Restaurant', 'Thai Restaurant', 'Comfort Food Restaurant', 'Cheese Shop', 'Stationery Store', 'Coworking Space', 'Latin American Restaurant', 'Bar', 'Gastropub', 'Bank', 'Gym / Fitness Center', 'Convenience Store', 'C

In [75]:

list_of_venue_categories.sort()
print ("The sorted list of {} venue categories is \n{}" .format(len(list_of_venue_categories), list_of_venue_categories))

The sorted list of 234 venue categories is 
['Afghan Restaurant' 'Airport' 'Airport Food Court' 'Airport Lounge'
 'Airport Service' 'Airport Terminal' 'American Restaurant' 'Antique Shop'
 'Aquarium' 'Art Gallery' 'Art Museum' 'Arts & Crafts Store'
 'Asian Restaurant' 'Athletics & Sports' 'Auto Workshop' 'BBQ Joint'
 'Baby Store' 'Bagel Shop' 'Bakery' 'Bank' 'Bar' 'Baseball Stadium'
 'Basketball Stadium' 'Beach' 'Bed & Breakfast' 'Beer Bar' 'Beer Store'
 'Belgian Restaurant' 'Bistro' 'Boat or Ferry' 'Bookstore' 'Boutique'
 'Brazilian Restaurant' 'Breakfast Spot' 'Brewery' 'Bubble Tea Shop'
 'Building' 'Burger Joint' 'Burrito Place' 'Bus Line' 'Butcher' 'Café'
 'Cajun / Creole Restaurant' 'Camera Store' 'Candy Store'
 'Caribbean Restaurant' 'Cheese Shop' 'Chinese Restaurant'
 'Chocolate Shop' 'Church' 'Climbing Gym' 'Clothing Store' 'Cocktail Bar'
 'Coffee Shop' 'College Arts Building' 'College Gym' 'College Rec Center'
 'Colombian Restaurant' 'Comfort Food Restaurant' 'Comic Shop'
 'Co

In [76]:
# count of venues grouped by Borough
venues_df.groupby(['Burough']).count()

,Neighborhood,Neigh_Lat,Neigh_Long,Venue_Cat,Venue_Name,Venue_Lat,Venue_Long
Burough,,,,,,,
Central Toronto,114,114,114,114,114,114,114
Downtown Toronto,1285,1285,1285,1285,1285,1285,1285
East Toronto,121,121,121,121,121,121,121
West Toronto,176,176,176,176,176,176,176


In [77]:
# count of venues grouped by Neighborhood
venues_df.groupby(['Neighborhood']).count()

,Burough,Neigh_Lat,Neigh_Long,Venue_Cat,Venue_Name,Venue_Lat,Venue_Long
Neighborhood,,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100,100
"Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14,14
Berczy Park,56,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17,17
"Cabbagetown, St. James Town",44,44,44,44,44,44,44
Central Bay Street,83,83,83,83,83,83,83
"Chinatown, Grange Park, Kensington Market",94,94,94,94,94,94,94
Christie,18,18,18,18,18,18,18


In [78]:
# count of venues grouped by Borough + Neighborhood
venues_df.groupby(['Burough','Neighborhood']).count()

Neigh_Lat  \
Burough          Neighborhood                                                    
Central Toronto  Davisville                                                 35   
                 Davisville North                                            9   
                 Deer Park, Forest Hill SE, Rathnelly, South Hil...         15   
                 Forest Hill North, Forest Hill West                         4   
                 Lawrence Park                                               3   
                 Moore Park, Summerhill East                                 2   
                 North Midtown, The Annex, Yorkville                        20   
                 North Toronto West                                         24   
                 Roselawn                                                    2   
Downtown Toronto Adelaide, King, Richmond                                  100   
                 Bathurst Quay, CN Tower, Harbourfront West, Isl...         14   
                 Berczy Park                                                56   
                 Cabbagetown, St. James Town                                44   
                 Central Bay Street                                         83   
                 Chinatown, Grange Park, Kensington Market                  94   
                 Christie                                                   18   
                 Church and Wellesley                                       88   
                 Commerce Court, Victoria Hotel                            100   
                 Design Exchange, Toronto Dominion Centre                  100   
                 First Canadian Place, Underground city                    100   
                 Garden District, Ryerson                                  100   
                 Harbord, University of Toronto                             36   
                 Harbourfront East, Toronto Islands, Union Station         100   
                 Harbourfront, Regent Park                                  51   
                 Rosedale                                                    4   
                 St. James Town                                            100   
                 Stn A PO Boxes 25 The Esplanade                            97   
East Toronto     Business Reply Mail Processing Centre 969 Eastern          17   
                 India Bazaar, The Beaches West                             19   
                 Riverdale, The Danforth West                               42   
                 Studio District                                            39   
                 The Beaches                                                 4   
West Toronto     Brockton, Exhibition Place, Parkdale Village               22   
                 Dovercourt Village, Dufferin                               15   
                 High Park, The Junction South                              24   
                 Little Portugal, Trinity                                   65   
                 Parkdale, Roncesvalles                                     15   
                 Runnymede, Swansea                                         35   

                                                                     Neigh_Long  \
Burough          Neighborhood                                                     
Central Toronto  Davisville                                                  35   
                 Davisville North                                             9   
                 Deer Park, Forest Hill SE, Rathnelly, South Hil...          15   
                 Forest Hill North, Forest Hill West                          4   
                 Lawrence Park                                                3   
                 Moore Park, Summerhill East                                  2   
                 North Midtown, The Annex, Yorkville                         20   
                 North Toronto West      

In [79]:
# count of venues grouped by Venue Category
venues_df.groupby(['Venue_Cat']).count()

,Burough,Neighborhood,Neigh_Lat,Neigh_Long,Venue_Name,Venue_Lat,Venue_Long
Venue_Cat,,,,,,,
Afghan Restaurant,1,1,1,1,1,1,1
Airport,1,1,1,1,1,1,1
Airport Food Court,1,1,1,1,1,1,1
Airport Lounge,2,2,2,2,2,2,2
Airport Service,3,3,3,3,3,3,3
Airport Terminal,2,2,2,2,2,2,2
American Restaurant,25,25,25,25,25,25,25
Antique Shop,2,2,2,2,2,2,2
Aquarium,5,5,5,5,5,5,5


In [80]:
venues_df.shape

(1696, 8)

In [81]:
venues_df.head()

,Burough,Neighborhood,Neigh_Lat,Neigh_Long,Venue_Cat,Venue_Name,Venue_Lat,Venue_Long
0,East Toronto,The Beaches,43.676357,-79.293031,Trail,Glen Manor Ravine,43.676821,-79.293942
1,East Toronto,The Beaches,43.676357,-79.293031,Health Food Store,The Big Carrot Natural Food Market,43.678879,-79.297734
2,East Toronto,The Beaches,43.676357,-79.293031,Pub,Grover Pub and Grub,43.679181,-79.297215
3,East Toronto,The Beaches,43.676357,-79.293031,Neighborhood,Upper Beaches,43.680563,-79.292869
4,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188,Greek Restaurant,Pantheon,43.677621,-79.351434


In [82]:
# one hot encoding
Borough_onehot = pd.get_dummies(venues_df[['Venue_Cat']], prefix="", prefix_sep="")
Borough_onehot.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
Borough_onehot.tail()

,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
1691,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1692,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1693,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1694,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1695,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
Borough_onehot.shape

(1696, 234)

In [85]:
# add borough column back to dataframe
Borough_onehot['Borough'] = venues_df['Burough'] 

# add neighborhood column back to dataframe
Borough_onehot['Neighborhood_1'] = venues_df['Neighborhood'] 

# move borough and neighborhood columns to the beginning
col_rearrange = [Borough_onehot.columns[-2]] + [Borough_onehot.columns[-1]] + list(Borough_onehot.columns[0:-2])
Borough_onehot = Borough_onehot[col_rearrange]

In [86]:
Borough_onehot.head()

,Borough,Neighborhood_1,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,East Toronto,The Beaches,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,East Toronto,The Beaches,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,East Toronto,The Beaches,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,East Toronto,The Beaches,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,East Toronto,"Riverdale, The Danforth West",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
Borough_grouped = Borough_onehot.groupby('Neighborhood_1').mean().reset_index()
Borough_grouped

,Neighborhood_1,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,...,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01,0.000000
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",0.000000,0.071429,0.071429,0.142857,0.214286,0.142857,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,0.000000,0.00,...,0.00000,0.00,0.012048,0.000000,0.000000,0.012048,0.000000,0.000000,0.00,0.012048
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00,0.042553,0.000000,0.053191,0.010638,0.000000,0.000000,0.00,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Church and Wellesley,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000,0.011364,0.000000,0.00,...,0.00000,0.00,0.000000,0.011364,0.011364,0.000000,0.000000,0.011364,0.00,0.011364


In [88]:
Borough_grouped.shape

(38, 235)

In [89]:
# PART 2 - CLUSTERS
kclusters = 4

In [90]:

Borough_grouped_clustering = Borough_grouped.drop('Neighborhood_1', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Borough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [91]:
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 2, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0], dtype=int32)

In [92]:
# add a column to include the Cluster for each row
Borough_grouped['Cluster'] = kmeans.labels_

In [93]:
Borough_grouped.shape

(38, 236)

In [94]:
    Borough_grouped['Neigh_Lat'] = np.nan
    Borough_grouped['Neigh_Long'] = np.nan
    Borough_grouped.head()

,Neighborhood_1,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster,Neigh_Lat,Neigh_Long
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,0.0,0.0,...,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0,NaN,NaN
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",0.0,0.071429,0.071429,0.142857,0.214286,0.142857,0.00,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0,NaN,NaN
2,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0,NaN,NaN
3,"Brockton, Exhibition Place, Parkdale Village",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0,NaN,NaN
4,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0,NaN,NaN


In [95]:
Borough_grouped.shape

(38, 238)

In [96]:
Borough_grouped.index.values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37])

In [97]:
venues_df.shape

(1696, 8)

In [98]:
venues_df.index.values

array([   0,    1,    2, ..., 1693, 1694, 1695])

In [99]:
for i  in range(Borough_grouped.shape[0]):
    temp_df = []
    temp_df = venues_df[venues_df['Neighborhood']==Borough_grouped.iloc[i]['Neighborhood_1']]
    temp_df = temp_df.iloc [:,1:4]
    for j  in range(1):                   # Just one execution only to get the Lat and Long for the neighborhood
        Borough_grouped.at[i, 'Neigh_Lat'] = temp_df.iloc[j]['Neigh_Lat']
        Borough_grouped.at[i, 'Neigh_Long'] = temp_df.iloc[j]['Neigh_Long']

In [ ]:
# create map of the neighborhoods with their clusters
map_clusters = folium.Map(location=[Borough_Latitude, Borough_Longitude], zoom_start=11)
# add markers to the map
markers_colors = ['red','blue','green','yellow']
fill_colors    = ['orange','purple','black','cyan']

for lat, lon, neigh, cluster in zip(
    Borough_grouped['Neigh_Lat'],
    Borough_grouped['Neigh_Long'],
    Borough_grouped['Neighborhood_1'],
    Borough_grouped['Cluster']):
    label = folium.Popup(str(neigh) + ' Cluster= ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=markers_colors[cluster-1],
        fill=True,
        fill_color=fill_colors[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [101]:
Borough_grouped.head()

,Neighborhood_1,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster,Neigh_Lat,Neigh_Long
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,0.0,0.0,...,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0,43.650571,-79.384568
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",0.0,0.071429,0.071429,0.142857,0.214286,0.142857,0.00,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0,43.628947,-79.394420
2,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0,43.644771,-79.373306
3,"Brockton, Exhibition Place, Parkdale Village",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0,43.636847,-79.428191
4,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0,43.662744,-79.321558


In [102]:
Borough_features_only = Borough_grouped.drop(['Neighborhood_1', 'Neigh_Lat', 'Neigh_Long', 'Cluster'],axis=1)
Borough_features_only.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,0.0,0.0,0.010000,...,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.0,0.01,0.0
1,0.0,0.071429,0.071429,0.142857,0.214286,0.142857,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.017857,...,0.0,0.0,0.017857,0.0,0.0,0.00,0.0,0.0,0.00,0.0
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0
4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0


In [103]:
# PART 3 - FEATURES
how_many_top_features_to_retain=5       

In [104]:
col_headings = ['Neigh']
col_headings_features = []
col_headings.append ('Lat')
col_headings.append ('Long')
col_headings.append ('Cluster')
for i in range(how_many_top_features_to_retain):
    if i==0:
        col_headings.append('1st Most Common Feature')
        col_headings_features.append('1st Most Common Feature')
    if i==1:
        col_headings.append('2nd MCF')
        col_headings_features.append('2nd MCF')
    if i==2:
        col_headings.append('3rd MCF')
        col_headings_features.append('3rd MCF')
    if i>2:
        col_headings.append(str(i+1)+'th MCF')
        col_headings_features.append(str(i+1)+'th MCF')      
print(col_headings)
print(col_headings_features)
df_neigh_combined=pd.DataFrame(columns=col_headings)
df_neigh_combined.head()

['Neigh', 'Lat', 'Long', 'Cluster', '1st Most Common Feature', '2nd MCF', '3rd MCF', '4th MCF', '5th MCF']
['1st Most Common Feature', '2nd MCF', '3rd MCF', '4th MCF', '5th MCF']


,Neigh,Lat,Long,Cluster,1st Most Common Feature,2nd MCF,3rd MCF,4th MCF,5th MCF


In [105]:
for i in range(0,Borough_grouped.shape[0]):
    df_1N_Neigh    = Borough_grouped.at[i,'Neighborhood_1']
    df_1N_Lat      = Borough_grouped.at[i,'Neigh_Lat']
    df_1N_Long     = Borough_grouped.at[i,'Neigh_Long']
    df_1N_Cluster  = Borough_grouped.at[i,'Cluster']
    #print("1. Intermediate result\n", i, df_1N_Neigh, df_1N_Lat, df_1N_Long, df_1N_Cluster)
    data_1N = Borough_features_only.iloc[i:i+1,:]
    #print("2. Intermediate result\n", data_1N)
    df_1N = ''
    df_1N = pd.DataFrame(data=data_1N,).T.reset_index()
    #print("3. Intermediate result\n", df_1N)
    df_1N.reset_index(drop=True,inplace=True)
    df_1N.rename(columns={i:'freq','index':'feature'}, inplace=True)
    df_1N.sort_values(['freq','feature'], ascending=[False,True], axis=0, inplace=True)
    #print("4. Intermediate result\n", df_1N)
    df_top_features_1N = df_1N.head(how_many_top_features_to_retain).T
    df_top_features_1N.columns=(col_headings_features)
    df_top_features_1N.drop('freq',axis=0,inplace=True)
    df_top_features_1N['Neigh']=df_1N_Neigh
    df_top_features_1N['Lat']  =df_1N_Lat
    df_top_features_1N['Long'] =df_1N_Long
    df_top_features_1N['Cluster'] = df_1N_Cluster
    df_top_features_1N.reset_index(drop=True,inplace=True)
    #print("5. Intermediate result\n", df_top_features_1N)
    df_neigh_combined = df_neigh_combined.append(df_top_features_1N,ignore_index=True)
    #print("6. Intermediate result\n", df_neigh_combined)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [106]:
df_neigh_combined.head(30)

,1st Most Common Feature,2nd MCF,3rd MCF,4th MCF,5th MCF,Cluster,Lat,Long,Neigh
0,Coffee Shop,Café,Bar,American Restaurant,Asian Restaurant,0,43.650571,-79.384568,"Adelaide, King, Richmond"
1,Airport Service,Airport Lounge,Airport Terminal,Airport,Airport Food Court,0,43.628947,-79.394420,"Bathurst Quay, CN Tower, Harbourfront West, Is..."
2,Coffee Shop,Bakery,Beer Bar,Café,Cheese Shop,0,43.644771,-79.373306,Berczy Park
3,Café,Breakfast Spot,Coffee Shop,Bakery,Bar,0,43.636847,-79.428191,"Brockton, Exhibition Place, Parkdale Village"
4,Light Rail Station,Auto Workshop,Brewery,Burrito Place,Comic Shop,0,43.662744,-79.321558,Business Reply Mail Processing Centre 969 Eastern
5,Coffee Shop,Bakery,Café,Italian Restaurant,Pharmacy,0,43.667967,-79.367675,"Cabbagetown, St. James Town"
6,Coffee Shop,Café,Ice Cream Shop,Italian Restaurant,Burger Joint,0,43.657952,-79.387383,Central Bay Street
7,Café,Bar,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,0,43.653206,-79.400049,"Chinatown, Grange Park, Kensington Market"
8,Café,Grocery Store,Coffee Shop,Park,Athletics & Sports,0,43.669542,-79.422564,Christie
9,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Burger Joint,Gay Bar,0,43.665860,-79.383160,Church and Wellesley


In [107]:
df_neigh_combined[df_neigh_combined['Cluster']==0]

,1st Most Common Feature,2nd MCF,3rd MCF,4th MCF,5th MCF,Cluster,Lat,Long,Neigh
0,Coffee Shop,Café,Bar,American Restaurant,Asian Restaurant,0,43.650571,-79.384568,"Adelaide, King, Richmond"
1,Airport Service,Airport Lounge,Airport Terminal,Airport,Airport Food Court,0,43.628947,-79.394420,"Bathurst Quay, CN Tower, Harbourfront West, Is..."
2,Coffee Shop,Bakery,Beer Bar,Café,Cheese Shop,0,43.644771,-79.373306,Berczy Park
3,Café,Breakfast Spot,Coffee Shop,Bakery,Bar,0,43.636847,-79.428191,"Brockton, Exhibition Place, Parkdale Village"
4,Light Rail Station,Auto Workshop,Brewery,Burrito Place,Comic Shop,0,43.662744,-79.321558,Business Reply Mail Processing Centre 969 Eastern
5,Coffee Shop,Bakery,Café,Italian Restaurant,Pharmacy,0,43.667967,-79.367675,"Cabbagetown, St. James Town"
6,Coffee Shop,Café,Ice Cream Shop,Italian Restaurant,Burger Joint,0,43.657952,-79.387383,Central Bay Street
7,Café,Bar,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,0,43.653206,-79.400049,"Chinatown, Grange Park, Kensington Market"
8,Café,Grocery Store,Coffee Shop,Park,Athletics & Sports,0,43.669542,-79.422564,Christie
9,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Burger Joint,Gay Bar,0,43.665860,-79.383160,Church and Wellesley


In [108]:
df_neigh_combined[df_neigh_combined['Cluster']==1]

,1st Most Common Feature,2nd MCF,3rd MCF,4th MCF,5th MCF,Cluster,Lat,Long,Neigh
32,Garden,Ice Cream Shop,Afghan Restaurant,Airport,Airport Food Court,1,43.711695,-79.416936,Roselawn


In [109]:
df_neigh_combined[df_neigh_combined['Cluster']==2]

,1st Most Common Feature,2nd MCF,3rd MCF,4th MCF,5th MCF,Cluster,Lat,Long,Neigh
26,Playground,Restaurant,Afghan Restaurant,Airport,Airport Food Court,2,43.689574,-79.38316,"Moore Park, Summerhill East"


In [110]:
df_neigh_combined[df_neigh_combined['Cluster']==3]

,1st Most Common Feature,2nd MCF,3rd MCF,4th MCF,5th MCF,Cluster,Lat,Long,Neigh
24,Bus Line,Park,Swim School,Afghan Restaurant,Airport,3,43.728020,-79.388790,Lawrence Park
31,Park,Playground,Trail,Afghan Restaurant,Airport,3,43.679563,-79.377529,Rosedale
